# Intro to SQL : Databases

- csv has downsides including: static, loadtime

## Querying
- SELECT : select without modification  
    -SELECT Rank,Major  
     FROM recent_grads;


In [ ]:
import sqlite3
conn = sqlite3.connect